In [1]:
!apt install aptitude swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [2]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
git is already installed at the requested version (1:2.17.1-1ubuntu0.7)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
The following NEW packages will be installed:
  file libmagic-mgc{a} libmagic1{a} libmecab-dev libmecab2{a} mecab mecab-ipadic{a} mecab-ipadic-utf8 mecab-jumandic{a} mecab-jumandic-utf8{a} mecab-utils{a} 
0 packages upgraded, 11 newly installed, 0 to remove and 31 not upgraded.
Need to get 29.3 MB of archives. After unpacking 282 MB will be used.
Get: 1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ub

In [3]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 209kB/s 


In [4]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [0]:
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [0]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
mecab = MeCab.Tagger("-d {0} -Owakati".format(path))
#mecab = MeCab.Tagger("-Owakati")

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import MeCab
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1
hidden_size = 256
 
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
 
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
mecab = MeCab.Tagger("-d {0} -Owakati".format(path))
 
#テキストの前処理
def ja_normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub('\r', '', s)
    s = re.sub('\n', '', s)
    s = re.sub(' ', ' ', s)
    s = mecab.parse(s)[:-2] #分かち書き
    return s #sentence

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
 
# Lowercase, trim, and remove non-letter characters
 
 
def en_normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def preprocessing(ss):
  return [ja_normalizeString(ss.split('\t')[0]), en_normalizeString(ss.split('\t')[1])]

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
 
    # lang1, lang2は言語の洗濯(lang1が英語、lang2がフランス)
    lines = open('drive/My Drive/dataset/nitiei_kihon/kyodai_ja2en.txt', encoding='utf-8').read().strip().split('\n')
 
    # l = 'eng ¥t french'
    # lに対してユニコードや正規化の処理
    #pairs = [[i1, o1], [i2, o2],....]
    pairs = [preprocessing(l) for l in lines]
    
    # 言語の翻訳の順番を変えるだけ
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        #初期化
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
 
    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 20 #文章の最大単語数(句読点を含む)
 
#上記2つの条件を満たすデータのみを抽出 
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
 
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [39]:
def prepareData(lang1, lang2, reverse=False):
    #テキストファイルを読んで行に分割し、行をペアに分割する
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse) 
    print("Read %s sentence pairs" % len(pairs))
    #データの選別
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    #単語の辞書を作る
    for pair in pairs:
        input_lang.addSentence(pair[0]) #英語の1文
        output_lang.addSentence(pair[1]) #フランス語の一文
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs
 
 
input_lang, output_lang, pairs = prepareData('ja', 'en')
print(len(pairs)) #shape->[10599, 2]
print(pairs[0])
print(random.choice(pairs))

Reading lines...
Read 5304 sentence pairs
Trimmed to 5241 sentence pairs
Counting words...
Counted words:
ja 7038
en 5333
5241
['x で は ない か と つくづく 疑問 に 思う', 'i often wonder if it might be x .']
['x が 日本 の 高度経済成長 を 支え た', 'x supported japan s rapid economic growth .']


In [0]:
"""
#入力データを反転させる
for i in range(len(pairs)):
  pairs[i][0] = ' '.join(pairs[i][0].split(' ')[::-1])
"""

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input_size：辞書の単語種類数、 hidden_size：分散表現の次元数
        #self.embedding = nn.Embedding.from_pretrained(embeddings=text_embedding_vectors, freeze=True)
        #self.emb_dim = text_embedding_vectors.shape[1]
        self.embedding = nn.Embedding(input_size, hidden_size)

        #層を増やす実験
        #self.gru1 = nn.GRU(text_embedding_vectors.shape[1], hidden_size)
        #self.lstm = nn.LSTM(text_embedding_vectors.shape[1],hidden_size) #[200, 256]
        self.lstm = nn.LSTM(hidden_size, hidden_size) #[200, 256]

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        #output, hidden2 = self.gru2(output, hidden2)
        #出力と隠れ状態
        return output, hidden
 
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
        #return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        #self.embedding = nn.Embedding.from_pretrained(embeddings=text_embedding_vectors, freeze=True)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        #self.gru1 = nn.GRU(self.hidden_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
 
    def forward(self, input, hidden, encoder_outputs):
        #input:[1], hidden:[1,1,256], encoder_outputs:[10,256]
        embedded = self.embedding(input).view(1, 1, -1) #[1, 1, 256]
        embedded = self.dropout(embedded)
        
        #加法注意によりattention_weightを求める。
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        
        #weightとエンコーダのアウトプットを用いる
        #bmmは各バッチごとに内積を出せる
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
 
        output = torch.cat((embedded[0], attn_applied[0]), 1) #[1, 512]
        output = self.attn_combine(output).unsqueeze(0) #[1, 1, 256]
 
        output = F.relu(output) #[1, 1, 256]
        output, hidden = self.lstm(output, hidden) #[1, 1, 256], ([1, 1, 256], [1, 1, 256])
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights
 
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
        #return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    #各文を単語indexのならびに
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1) #[[2], [25], [9]....]の形に


def tensorsFromPair(pair):
    #pairは各文(0がフランス)
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5
 
 
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
 
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
 
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
 
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
 
    loss = 0
    #input_tensorは[その文の単語数, 1]のテンソル、つまり1文のデータ
    for ei in range(input_length):
        #1文字のindexと隠れ状態を入れていく
        #input_tensor[ei]:[1], encoder_hidden:[1,1,256]
        encoder_output, encoder_hidden, = encoder(
            input_tensor[ei], encoder_hidden)
        #encoder_output:[1,1,256], encoder_hidden:[1,1,256]
        encoder_outputs[ei] = encoder_output[0, 0]
 
    decoder_input = torch.tensor([[SOS_token]], device=device)
 
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            #decoder_input:[1], decoder_hidden:[1,1,256], encoder_outputs:[10,256]
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            #decoder_output:[1,2803], decoder_hidden:[1,1,256], decoder_attention:[1,10]
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
 
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
 
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
 
    loss.backward()
 
    encoder_optimizer.step()
    decoder_optimizer.step()
 
    return loss.item() / target_length

In [0]:
import time
import math
 
 
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
 
 
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
 
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
 
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        #########
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        #一文の損失をだす
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion) 
        print_loss_total += loss
        plot_loss_total += loss

        #5000回で表示
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every #5000会のlossの平均
            ppl = math.exp(print_loss_avg) #pplをだす
            print_loss_total = 0
            print('%s (%d %d%%) %.4f %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg, ppl))
 
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
 
    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
 
 
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
 
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
 
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
 
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
 
        decoder_hidden = encoder_hidden
        
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
 
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<eos>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
 
            decoder_input = topi.squeeze().detach()
 
        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        #pair = pairs[i]
        pair = random.choice(pairs)
        print('入力：', pair[0])
        print('正解', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('応答：', output_sentence)
        print('')

In [50]:
output_lang.n_words

5333

In [62]:
"""
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, ).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
"""

1m 57s (- 27m 31s) (5000 6%) 4.8435 126.9158
3m 55s (- 25m 28s) (10000 13%) 4.6635 106.0052
5m 53s (- 23m 33s) (15000 20%) 4.4511 85.7215
7m 51s (- 21m 35s) (20000 26%) 4.2701 71.5277
9m 49s (- 19m 39s) (25000 33%) 4.1124 61.0958
11m 46s (- 17m 40s) (30000 40%) 3.9269 50.7471
13m 43s (- 15m 40s) (35000 46%) 3.7267 41.5428
15m 40s (- 13m 42s) (40000 53%) 3.5254 33.9686
17m 37s (- 11m 45s) (45000 60%) 3.3206 27.6774
19m 35s (- 9m 47s) (50000 66%) 3.0889 21.9528
21m 33s (- 7m 50s) (55000 73%) 2.8810 17.8329
23m 32s (- 5m 53s) (60000 80%) 2.6869 14.6858
25m 32s (- 3m 55s) (65000 86%) 2.5326 12.5867
27m 32s (- 1m 58s) (70000 93%) 2.3667 10.6625
29m 30s (- 0m 0s) (75000 100%) 2.1292 8.4079


In [64]:
trainIters(encoder1, attn_decoder1, 10000, print_every=5000)

1m 59s (- 1m 59s) (5000 50%) 1.9541 7.0578
3m 59s (- 0m 0s) (10000 100%) 1.8223 6.1863


In [63]:
evaluateRandomly(encoder1, attn_decoder1)

入力： ふいに 山田 の 声 が 私 の 耳 に 届く
正解 suddenly i heard mr . ms . yamada s voice .
応答： mr . yamada . yamada at my at at . <eos>

入力： 彼 が 作品 を ３分 ほど の 高画質 動画 で 紹介 し ます
正解 he will introduce the work in a high definition video that s about three minutes long .
応答： he will about the the as as as he was about . . . <eos>

入力： 彼 は 牧場 で のんびり 余生 を 送り ます
正解 he spends his remaining time leisurely out in the pasture .
応答： he spends his remaining time the at the in . . . . <eos>

入力： 彼 が 動画配信 サイト の デザイン ・ 制作 業務 を 担当 し ます
正解 he is in charge of the design and production of the video distribution website .
応答： he is the charge of the and and and the of the . . . . . <eos>

入力： x が 、 何とも 言え ない
正解 x is indescribable .
応答： x is not . . <eos>

入力： とにかく 商品 の 種類 が 多い
正解 anyway it has a wider variety of items .
応答： there are a items items . <eos>

入力： 彼 が わざわざ 訊き に 来る 筈 が 無い
正解 he has never come all the way out here to ask about that .
応答： he has t at all all the at . . . . <eos>

入力： エンジニア 達 の 目 が しっかり 前 を 向く
正解 t

In [65]:
trainIters(encoder1, attn_decoder1, 50000, print_every=5000)

2m 1s (- 18m 14s) (5000 10%) 1.6552 5.2340
4m 1s (- 16m 4s) (10000 20%) 1.5347 4.6400
6m 2s (- 14m 5s) (15000 30%) 1.3848 3.9941
8m 4s (- 12m 7s) (20000 40%) 1.2619 3.5322
10m 4s (- 10m 4s) (25000 50%) 1.1133 3.0443
12m 3s (- 8m 2s) (30000 60%) 1.0312 2.8045
14m 2s (- 6m 1s) (35000 70%) 0.9307 2.5363
16m 1s (- 4m 0s) (40000 80%) 0.8330 2.3001
18m 4s (- 2m 0s) (45000 90%) 0.7773 2.1756
20m 4s (- 0m 0s) (50000 100%) 0.6998 2.0134


In [66]:
evaluateRandomly(encoder1, attn_decoder1)

入力： x を ほとんど の 方 が 御存知 だ
正解 most people know about x .
応答： most people know about x . <eos>

入力： x は 時間 の 問題 と 思い ます
正解 i think x is just a matter of time .
応答： i think x is in it s is . . <eos>

入力： ｒｓｓ で サイト の 更新 情報 を 配信 し ます
正解 we will deliver updates in an rss feed .
応答： we will deliver updates in an life . <eos>

入力： 手元 に ほとんど 写真 が 無い
正解 i have few pictures on hand .
応答： i have few pictures on the . <eos>

入力： x に は あまり 興味 が 無い
正解 i m not very interested in x .
応答： i m not very interested in x . <eos>

入力： x を 心から 楽しみ に し て い ます
正解 i m really looking forward to x .
応答： i m really looking forward to x . <eos>

入力： あなた は いつも 周囲 の 人たち を 驚か せ ます
正解 you always surprise people around you .
応答： you always surprise people around you . <eos>

入力： 私 は ダブル 洗顔 で しっかり 毛穴 の 汚れ を 落とし ます
正解 i wash my face twice to remove pore clogging dirt .
応答： i wash my face twice twice at it . <eos>

入力： それ が 多く の 仲間 を 増やし まし た
正解 it has increased the number of friends .
応答： it has increased the number of its 

In [67]:
trainIters(encoder1, attn_decoder1, 10000, print_every=5000)

2m 1s (- 2m 1s) (5000 50%) 0.6408 1.8980
4m 1s (- 0m 0s) (10000 100%) 0.5514 1.7356


In [68]:
evaluateRandomly(encoder1, attn_decoder1)

入力： 油断 が 原因 だ
正解 carelessness is the reason .
応答： anger is the reason . <eos>

入力： 彼 が x を 目標 に し ます
正解 he regards x as an objective .
応答： he regards x as an objective . <eos>

入力： スタッフ が 只今 、 １２月 の 予約 を 承っ て おり ます
正解 our staff is now accepting reservations for december .
応答： our staff is now accepting reservations for december . <eos>

入力： あなた は いつも 周囲 の 人たち を 驚か せ ます
正解 you always surprise people around you .
応答： you always people around you you around you . <eos>

入力： 彼 が ふと 時計 に 目 を やる
正解 he glances at his watch .
応答： he glances at his watch . <eos>

入力： 申請 者 が 自ら 市長 に 暗証番号 の 登録 を 申請 する
正解 the applicants themselves apply for a pin registration from the mayor .
応答： the applicants themselves apply for a pin registration from the mayor . <eos>

入力： さらに ビタミン ｃ の 効果 が アップ し ます
正解 the effects of vitamin c are enhanced .
応答： the effects of vitamin c are enhanced . <eos>

入力： 是非 更新 情報 の 登録 を お願い し ます
正解 please update the registered information .
応答： please update the registered information

In [0]:
import pickle

model_path = 'drive/My Drive/model/ja2en_encoder0519.pth'
torch.save(encoder1.to('cpu').state_dict(), model_path)
model_path = 'drive/My Drive/model/ja2en_decoder0519.pth'
torch.save(attn_decoder1.to('cpu').state_dict(), model_path)

with open('drive/My Drive/model/ja2en_input_word2index0519.pkl', 'wb') as f:
    pickle.dump(input_lang.word2index, f)
with open('drive/My Drive/model/ja2en_input_index2word0519.pkl', 'wb') as f:
    pickle.dump(input_lang.index2word, f)
with open('drive/My Drive/model/ja2en_output_index2word0519.pkl', 'wb') as f:
    pickle.dump(output_lang.index2word, f)
with open('drive/My Drive/model/ja2en_output_word2index0519.pkl', 'wb') as f:
    pickle.dump(output_lang.word2index, f)